In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[8]')
    .config('spark.driver.memory', '16g')
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/10 03:31:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
train_data = spark.read.parquet('../data/amex-default-prediction/train_data')
train_labels = spark.read.parquet('../data/amex-default-prediction/train_labels')

In [3]:
train_labels.limit(10).show()

+--------------------+------+
|         customer_ID|target|
+--------------------+------+
|0000099d6bd597052...|   0.0|
|00000fd6641609c6e...|   0.0|
|00001b22f846c82c5...|   0.0|
|000041bdba6ecadd8...|   0.0|
|00007889e4fcd2614...|   0.0|
|000084e5023181993...|   0.0|
|000098081fde4fd64...|   0.0|
|0000d17a1447b25a0...|   0.0|
|0000f99513770170a...|   1.0|
|00013181a0c5fc8f1...|   1.0|
+--------------------+------+



In [4]:
train_labels.groupBy('customer_ID').count().orderBy('count', ascending=False).limit(10).show()

+--------------------+-----+
|         customer_ID|count|
+--------------------+-----+
|0012e41fe6caa3ba3...|    1|
|00582971eb87cf0bb...|    1|
|00dbda372d97f2357...|    1|
|00dc3b4e9f8f3b114...|    1|
|0119d38628b2cbdd0...|    1|
|0125e11ed5c94d63c...|    1|
|01439ee3abf1b4552...|    1|
|014acecc038c204f2...|    1|
|01500e2a9f82cfab4...|    1|
|0152860993fe1a7b1...|    1|
+--------------------+-----+



In [5]:
train_data.limit(10).toPandas()

22/06/10 03:31:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
/usr/local/lib/python3.8/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.8/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.8/dist-packages/pyspark/sql/pandas/

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00403d34ba512d4c72dc6fd805b84498d0dcadcfbb58ca...,2018-03-21,0.791787,0.683767,0.035230,1.004302,0.003275,0.101050,0.008144,0.003354,...,NaN,NaN,NaN,0.006559,0.002691,0.003431,NaN,0.000187,0.000034,0.002045
1,00b50b0ab89bfb8ef1a5942b18bea9994d0c3207131e56...,2017-07-29,0.943163,0.529789,0.175606,1.004728,0.004517,0.298369,0.003098,0.003244,...,NaN,NaN,NaN,0.005845,0.005304,0.008459,NaN,0.008706,0.005755,0.001480
2,004cf587f8db6d3edb18bce27c6c4855e7c117ef8b2501...,2017-10-25,0.825849,0.009611,0.022143,1.005095,0.004939,0.152467,0.001338,0.001610,...,NaN,NaN,NaN,0.005062,0.009416,0.008604,NaN,0.005272,0.004203,0.003022
3,0216684b0b1faf80959e5c9c8e14c80e2b10eb851b573b...,2017-09-21,0.848227,0.008164,0.002402,0.815999,0.002968,NaN,0.005372,0.003133,...,NaN,NaN,NaN,0.006897,0.002919,0.009751,NaN,0.003399,0.001608,0.002856
4,01a279b5e7dafb8970911f74da31f0a1837af0709dfc49...,2017-11-28,0.901389,0.004876,0.008624,0.819405,0.006558,NaN,0.007316,0.004203,...,NaN,NaN,NaN,0.002448,0.003578,0.007411,NaN,0.003352,0.009872,0.001309
5,0071b1b3cb9efec4a628fc21ac143ef92d0fe6b58dbf71...,2017-10-08,0.859257,0.001360,0.005847,1.006282,0.000465,0.416519,0.009725,0.009772,...,NaN,NaN,NaN,0.000938,0.007070,0.007457,NaN,0.006416,0.002880,0.005012
6,01ae75485b7d790e0adcdc8d60dfdf5898b2dbdf67384d...,2017-10-14,0.326465,0.006871,0.077844,0.254774,0.008865,0.788595,0.008406,0.043803,...,NaN,NaN,NaN,0.007373,0.001529,0.005971,NaN,0.000473,0.009077,0.002814
7,003b8acbe4a0440cc0793413cb667ff2aea374ff675c67...,2017-12-23,0.881174,0.239852,0.036496,1.000403,0.015892,0.118295,0.008450,0.000240,...,NaN,NaN,NaN,0.001219,0.005738,0.003018,NaN,0.000663,0.008475,0.007551
8,0178e40e713acd4367d67a0b5d4451753edb7df39ea7b7...,2017-05-13,0.275334,0.006802,0.003530,0.814972,0.008282,NaN,0.009127,0.006758,...,NaN,NaN,NaN,0.001818,0.003369,0.008177,NaN,0.003823,0.004214,0.005009
9,01b09a1d445a311a100c90f3a4a3088127bdcc96b99b79...,2017-06-23,0.958853,0.003488,0.007762,0.815809,0.000493,NaN,0.009705,0.008903,...,NaN,NaN,NaN,0.008910,0.000923,0.008609,NaN,0.002536,0.000206,0.008547


In [6]:
import pyspark.sql.functions as F
from datetime import timedelta
from pyspark.sql.types import ArrayType, FloatType


def diff_days(list_of_dates):
    list_of_dates = sorted(list_of_dates)
    out = []
    for i in range(1, len(list_of_dates)):
        out.append((list_of_dates[i] - list_of_dates[i-1]) / timedelta(days=1))
    return out

metrics_aggregated_by_id = (
    train_data
    .groupBy('customer_ID')
    .agg(
        F.collect_list('S_2').alias('S_2_list'),
        F.sum(F.lit(1)).alias('num_rows'),
        F.min('S_2').alias('min_date'),
        F.max('S_2').alias('min_date'),
    )
    .withColumn('diff_days', F.udf(diff_days, ArrayType(FloatType()))('S_2_list'))
    .drop('S_2_list')
)


In [7]:
import pandas as pd
import numpy as np

probabilities = list(np.linspace(0, 1, num=20))
diff_days_quantiles = (
    metrics_aggregated_by_id
    .withColumn('diff_days', F.explode('diff_days'))
    .approxQuantile(col='diff_days', probabilities=probabilities, relativeError=.01)
)
pd.DataFrame({
    'probabilities': probabilities,
    'diff_days_quantiles': diff_days_quantiles,
})

,probabilities,diff_days_quantiles
0,0.000000,1.0
1,0.052632,12.0
2,0.105263,17.0
3,0.157895,20.0
4,0.210526,24.0
5,0.263158,26.0
6,0.315789,28.0
7,0.368421,29.0
8,0.421053,30.0
9,0.473684,30.0


In [8]:
num_rows_quantiles = metrics_aggregated_by_id.approxQuantile(col='num_rows', probabilities=probabilities, relativeError=.01)
pd.DataFrame({
    'probabilities': probabilities,
    'num_rows_quantiles': num_rows_quantiles,
})

,probabilities,num_rows_quantiles
0,0.000000,1.0
1,0.052632,5.0
2,0.105263,9.0
3,0.157895,13.0
4,0.210526,13.0
5,0.263158,13.0
6,0.315789,13.0
7,0.368421,13.0
8,0.421053,13.0
9,0.473684,13.0
